In [1]:
import inspire_agrivolt

import xarray as xr
import pvdeg

from dask.distributed import Client, LocalCluster

In [2]:
loaded = xr.open_zarr("/projects/inspire/PySAM-MAPS/Full-Outputs/colorado/04.zarr")

sub_loaded = loaded.isel(gid=slice(0,10))

### get associated nsrdb metadata
weather_db = "NSRDB"
weather_arg = {
    "satellite": "Americas",
    "names": "TMY",
    "NREL_HPC": True,
}

geo_weather, geo_meta = pvdeg.weather.get(
    weather_db, geospatial=True, **weather_arg)

sub_meta = geo_meta.loc[sub_loaded.gid.values]

/home/tford/.conda-envs/geospatial/lib/python3.9/site-packages/xarray/core/dataset.py:277: UserWarning: The specified chunks separate the stored chunks along dimension "phony_dim_1" starting at index 500. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


In [3]:
conf = "04"

template = inspire_agrivolt.beds_postprocessing.output_template_gid(
    sub_loaded,
    conf=conf
)

template_mash, sub_loaded_mash = xr.unify_chunks(template, sub_loaded)

res = pvdeg.geospatial.analysis(
    weather_ds = sub_loaded_mash,
    meta_df = sub_meta,
    func=inspire_agrivolt.beds_postprocessing.testbeds_irradiance,
    template=template_mash,

    # kwargs passed to func
    conf=conf
)

{'underpanel': time
2001-06-11 21:00:00    17.19125
2001-06-11 21:00:00    17.19125
2001-06-11 21:00:00    17.19125
2001-06-11 21:00:00    17.19125
2001-06-11 21:00:00    17.19125
                         ...   
2001-06-20 15:00:00     4.96125
2001-06-20 15:00:00     4.96125
2001-06-20 15:00:00     4.96125
2001-06-20 15:00:00     4.96125
2001-06-20 15:00:00     4.96125
Length: 21100, dtype: float64, 'edgetoedge': time
2001-06-11 21:00:00    83003.506015
2001-06-11 21:00:00    83003.558646
2001-06-11 21:00:00    83003.611278
2001-06-11 21:00:00    83003.663909
2001-06-11 21:00:00    83003.716541
2001-06-11 21:00:00    83003.769172
dtype: float64, 'bedA': time
2001-06-11 21:00:00    179683.539507
2001-06-11 21:00:00    179683.539507
2001-06-11 21:00:00    179683.539507
2001-06-11 21:00:00    179683.539507
2001-06-11 21:00:00    179683.539507
                           ...      
2001-06-20 15:00:00    179683.539507
2001-06-20 15:00:00    179683.539507
2001-06-20 15:00:00    179683.539507


ValueError: cannot reindex on an axis with duplicate labels

In [ ]:
template

In [ ]:
sub_loaded